# Complete Evolutionary Algorithm TSP Solution

This notebook presents a comprehensive implementation of the Traveling Salesman Problem (TSP) using an Evolutionary Algorithm (EA) approach with Italian cities data. All required libraries, functions, and logging for complete functionality are included.

In [ ]:

import numpy as np
import pandas as pd
from geopy.distance import geodesic
import logging
import random
from dataclasses import dataclass
from itertools import combinations

# Set up logging
logging.basicConfig(level=logging.DEBUG)


In [ ]:

# Load city data and set up distance matrix
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['City', 'Latitude', 'Longitude'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.Latitude, c1.Longitude), (c2.Latitude, c2.Longitude)).kilometers


In [ ]:

# EA parameters
population_size = 100
num_generations = 1000
mutation_rate = 0.01

@dataclass
class RouteCandidate:
    path: list
    distance: float = None

# Calculate the cost of a route
def route_distance(candidate: RouteCandidate, dist_matrix):
    if candidate.distance is None:
        candidate.path.append(candidate.path[0])  # Close the cycle
        candidate.distance = sum(dist_matrix[candidate.path[i], candidate.path[i + 1]] 
                                 for i in range(len(candidate.path) - 1))
        candidate.path.pop()
    return candidate.distance

# Randomized parent selection with roulette wheel
def roulette_selection(population, fitness_scores):
    max_score = sum(fitness_scores)
    pick = random.uniform(0, max_score)
    current = 0
    for i, score in enumerate(fitness_scores):
        current += score
        if current > pick:
            return population[i]
    return population[-1]

# Mutation by shuffling elements
def shuffle_mutation(indiv: RouteCandidate):
    mutated_path = indiv.path[:]
    i, j = random.sample(range(len(mutated_path)), 2)
    mutated_path[i], mutated_path[j] = mutated_path[j], mutated_path[i]
    return RouteCandidate(mutated_path)

# Crossover by combining segments from parents
def crossover_segments(parent1: RouteCandidate, parent2: RouteCandidate):
    size = len(parent1.path)
    start, end = sorted(random.sample(range(size), 2))
    child_path = [-1] * size
    child_path[start:end] = parent1.path[start:end]
    
    pos = end
    for city in parent2.path:
        if city not in child_path:
            if pos == size:
                pos = 0
            child_path[pos] = city
            pos += 1
    return RouteCandidate(child_path)

# EA main function
def execute_ea_algorithm(dist_matrix, city_count, population_size, generations):
    # Initialize population
    population = [RouteCandidate(path=random.sample(range(city_count), city_count)) for _ in range(population_size)]
    best_solution = min(population, key=lambda x: route_distance(x, dist_matrix))
    
    for generation in range(generations):
        fitness_scores = [1 / route_distance(ind, dist_matrix) for ind in population]
        new_population = []
        
        for _ in range(population_size // 2):
            parent1 = roulette_selection(population, fitness_scores)
            parent2 = roulette_selection(population, fitness_scores)
            offspring1 = crossover_segments(parent1, parent2)
            offspring2 = crossover_segments(parent2, parent1)
            
            if random.random() < mutation_rate:
                offspring1 = shuffle_mutation(offspring1)
            if random.random() < mutation_rate:
                offspring2 = shuffle_mutation(offspring2)
            
            new_population.extend([offspring1, offspring2])
        
        population = new_population
        current_best = min(population, key=lambda x: route_distance(x, dist_matrix))
        if route_distance(current_best, dist_matrix) < route_distance(best_solution, dist_matrix):
            best_solution = current_best

    best_route = [CITIES.iloc[i]['City'] for i in best_solution.path]
    return best_route, route_distance(best_solution, dist_matrix)

# Run EA
optimal_route, min_cost = execute_ea_algorithm(DIST_MATRIX, len(CITIES), population_size, num_generations)
logging.info(f"Optimal Route: {optimal_route}")
logging.info(f"Total Distance: {min_cost:.2f} km")
